In [1]:
le = [{'uuid': '7e8502d8-be78-457a-a28c-94db9db40f5e', 'id': 11400}, {'uuid': '0e1288d7-c2de-4cd2-bb0b-287d080536f2', 'id': 11401}, {'uuid': 'ca7c6a75-5eae-4aac-ba59-284872ec4ee6', 'id': 11402}, {'uuid': '338b9e9a-cc8d-4e85-b44d-fb600cb524dc', 'id': 11403}, {'uuid': '1ac7b396-b064-43fe-a1b8-6e83fa7d08c1', 'id': 11404}, {'uuid': '60a98ac8-e973-4a96-912a-7ccde22b274c', 'id': 11405}, {'uuid': '30016d23-6593-41d0-acc8-ee1c62afed79', 'id': 11406}, {'uuid': 'de60088d-7522-4fd7-8688-d54d4fe06910', 'id': 11407}, {'uuid': 'e5fee915-f535-40d6-a6c4-f9de2d4b8167', 'id': 11408}, {'uuid': 'f476ba69-7197-4609-823f-4201598645aa', 'id': 11409}, {'uuid': 'ed50c838-3535-4634-8db3-58e7980d5d83', 'id': 11410}, {'uuid': 'db962be3-d75c-43a3-a769-85a845a8c2f8', 'id': 11411}, {'uuid': 'a5f45cb0-62ca-4569-9ae2-331a2a2497a8', 'id': 11412}, {'uuid': '3528b549-a081-4d14-bbd6-538c74dc7406', 'id': 11413}, {'uuid': 'fe37db02-2946-4f9c-b8e2-8e995a24b39a', 'id': 11414}, {'uuid': '832b7c16-7edf-45b9-8ca7-1f4f0aad2b98', 'id': 11415}, {'uuid': '2272400a-7981-4e92-a0b6-3702956994eb', 'id': 11416}, {'uuid': '396d30f4-203b-4fa3-b80a-aa08abfca641', 'id': 11417}, {'uuid': 'd3ab5c28-92f6-42a6-9d46-8cf10665e127', 'id': 11418}, {'uuid': '780c0c53-b782-43c3-a685-b1db4b0ad622', 'id': 11419}, {'uuid': '49f1e2e4-5ca6-4c4b-a133-8950b41cdde0', 'id': 11420}, {'uuid': '69c32b1e-2938-4f15-be3c-599f61c72b08', 'id': 11421}, {'uuid': '99c8d95f-875c-4949-8e93-63861e19e776', 'id': 11422}, {'uuid': '2fcc8b09-b80f-40a1-97e6-a61a6ddfd1b5', 'id': 11423}, {'uuid': '3c111105-d6d5-414b-9ac2-2e46d647d1cd', 'id': 11424}, {'uuid': '71a0e6f4-fde7-477e-93f4-65e9e531f90c', 'id': 11425}, {'uuid': '2746a1df-d463-4459-a0ad-b64a5f9261ac', 'id': 11426}, {'uuid': '50676155-ca0c-405e-b6a9-1b0124908691', 'id': 11427}, {'uuid': 'cd0c9e66-f30d-4a6e-bc56-19903f4f70f1', 'id': 11428}, {'uuid': 'e8ece8cc-f8a8-44a1-bdbf-0633da21e5de', 'id': 11429}, {'uuid': 'b7075c61-9925-4b0f-960a-ec5684ed35d5', 'id': 11430}, {'uuid': 'adc830d6-d0ab-44cc-8215-94932eca2dd5', 'id': 11431}, {'uuid': '2d6b122b-a093-47bc-8b36-1bc9178ca745', 'id': 11432}, {'uuid': '4baa3bd4-b245-44d6-b53e-5d2a6772eef7', 'id': 11433}, {'uuid': 'f87aae9d-69be-449b-a8e5-9b5bfe796066', 'id': 11434}, {'uuid': 'fd4eb5c5-372e-40bb-a2a3-6f40e897c0e8', 'id': 11435}, {'uuid': '118c5678-9c98-4f14-8093-88ce94485d72', 'id': 11436}, {'uuid': '6d005fad-0170-4e52-8dd3-3ddf4aa92c88', 'id': 11437}, {'uuid': 'ec352d1d-bfac-46a9-a4a5-a2c3d9fef156', 'id': 11438}, {'uuid': '246ca3fd-8572-4854-b817-aead6e31a519', 'id': 11439}, {'uuid': '825b58a6-e728-4e37-bfc2-c2947c93ba70', 'id': 11440}, {'uuid': '352c0221-b19a-4524-980a-3c4793962e63', 'id': 11441}, {'uuid': '2b2ddb13-4c32-4089-bb02-08f13a8c5ff6', 'id': 11442}, {'uuid': '0426dc39-c615-43c2-9f1e-58930a4787a1', 'id': 11443}, {'uuid': '66821a49-4de6-4257-85ce-98b73d8c9fc3', 'id': 11444}, {'uuid': 'bdd968e0-6111-4a2d-a148-1937af190009', 'id': 11445}, {'uuid': '4b7ec781-f4ae-4a60-b24c-79f39ceee5b5', 'id': 11446}, {'uuid': 'b0f67114-a14e-4267-b34b-654c98eca48b', 'id': 11447}, {'uuid': '64bfe399-4f60-4711-aaca-5347fc1fa1ab', 'id': 11448}, {'uuid': 'd9613d52-c822-40bd-8603-088791ca08c5', 'id': 11449}, {'uuid': '08a034e9-5dc6-463a-b8dd-ab476aaabbf7', 'id': 11450}, {'uuid': '7a81167b-aafd-4376-961b-c0f0585760b0', 'id': 11451}, {'uuid': '492d5ab8-78fe-4dd6-91bd-31c4390637b4', 'id': 11452}, {'uuid': '01324e80-808a-4460-b8b2-767c879304bb', 'id': 11453}, {'uuid': '533df7ef-cdc9-4336-84a8-e771b2786bf1', 'id': 11454}, {'uuid': '45911caf-a93e-4766-83a8-93bfb0a44c6b', 'id': 11455}, {'uuid': '17637dad-f05d-4844-a335-8eb528e6a4eb', 'id': 11456}, {'uuid': '5e782a4a-79b7-479a-bac0-ba5c2be09478', 'id': 11457}, {'uuid': '9f2c18aa-6fa8-4984-9446-16547f457f87', 'id': 11458}, {'uuid': '3e00b00a-f819-44d4-bdcf-eb45cc84cf14', 'id': 11459}, {'uuid': '79ba114e-1f25-44e1-b429-e7b54a7fda1b', 'id': 11460}, {'uuid': '47c328d4-fe6d-4a8b-a58e-34ef05a2ff47', 'id': 11461}, {'uuid': 'dd1b50f0-8d02-4d16-a84b-91ad5119f302', 'id': 11462}, {'uuid': 'f1dabe14-8198-4d35-9455-946a6af588ad', 'id': 11463}, {'uuid': 'eb42702a-5444-4a08-96b1-115af519b59a', 'id': 11464}, {'uuid': '2d01c52f-b1f7-4869-be7a-43edb90a43b8', 'id': 11465}, {'uuid': '98efb941-7434-4249-9db1-90d1dacc3efb', 'id': 11466}, {'uuid': '5c784ad1-fd51-4151-b4d7-5a2a77555416', 'id': 11467}, {'uuid': '315c8013-b745-4415-94fd-cbaa09e901db', 'id': 11468}, {'uuid': '8b987254-fc78-43df-acc6-a1fcc889ab6f', 'id': 11469}, {'uuid': 'ecbb66e9-69ca-426d-9a58-754e6869d61d', 'id': 11470}, {'uuid': '384f66ea-7079-479a-8b66-07453f73fd26', 'id': 11471}, {'uuid': '0bdb06ba-7e0b-4623-8752-835ef86a6d03', 'id': 11472}, {'uuid': '20b69578-86f1-428f-9a89-32d1a7977d39', 'id': 11473}, {'uuid': '4b8d9b45-b3bb-4358-b331-f6f5856579fa', 'id': 11474}, {'uuid': 'a6600741-77c2-4c5e-8f8f-37ae55526a7a', 'id': 11475}, {'uuid': '0c71e6b3-348b-4a60-bf73-d409e8faa084', 'id': 11476}, {'uuid': '9b76a19a-604d-424d-a538-037515eef236', 'id': 11477}, {'uuid': '6208ef9f-4a28-48f8-a3bc-633b458a86cf', 'id': 11478}, {'uuid': 'f5d40123-cf3a-4e5c-bbcb-f8668e831d97', 'id': 11479}, {'uuid': '6988fb32-a93c-4921-88d9-70e6d99cd4d6', 'id': 11480}, {'uuid': '1f675528-e5dd-428a-899e-9675c04a8793', 'id': 11481}, {'uuid': 'bb252624-e1c2-4143-808c-cb4683efa7ad', 'id': 11482}, {'uuid': '274872c5-49fb-458c-89ad-07ba4b6438d8', 'id': 11483}, {'uuid': '8593ce95-bab1-457d-9565-ce9044966015', 'id': 11484}, {'uuid': '3e1d6bb3-005d-4407-a92f-98e8f8cc943c', 'id': 11485}, {'uuid': 'f001ba28-df65-489e-8863-1245f84aabad', 'id': 11486}, {'uuid': '97b31ce0-32b0-4cb6-966c-a365235857de', 'id': 11487}, {'uuid': '8887fde9-f407-4f1b-8c9d-06fea9de096f', 'id': 11488}, {'uuid': 'da3fd6b0-5e62-40d3-a78d-5ad65d7ca40d', 'id': 11489}, {'uuid': 'c11d74d1-56ec-4de4-90e3-b833b8034dc4', 'id': 11490}, {'uuid': 'c772bb4e-f66a-41e6-9bb3-692ac7d9292a', 'id': 11491}, {'uuid': '845dd065-4ae4-404b-b546-0ea05dbab0a2', 'id': 11492}, {'uuid': 'f1c364a0-d131-40e8-8fb8-bd3ca76c0ec8', 'id': 11493}, {'uuid': '9b27dc4c-75bf-4424-b8de-bd2ed5186ead', 'id': 11494}, {'uuid': '7a22066f-df33-42c9-a132-8c7eaaa912dc', 'id': 11495}, {'uuid': '2e3192bd-6233-4f03-b469-2756fa8e7647', 'id': 11496}, {'uuid': '4fffbb4f-2e73-4bb9-8586-27d40455f7b7', 'id': 11497}, {'uuid': 'fcbd8ba1-7723-430a-87cb-c439f2bb5695', 'id': 11498}, {'uuid': '3f40d278-1be5-4537-867a-277a44369bf9', 'id': 11499}, {'uuid': 'a5962cd4-560f-4301-a3f0-7441fb385382', 'id': 11500}, {'uuid': '4bca7d21-ea77-44ce-aca1-79f3d5f57ec5', 'id': 11501}, {'uuid': 'f016a3f5-d9b8-4f1b-9040-5d33419665b8', 'id': 11502}, {'uuid': '668cb697-a009-4b44-a638-a4b7857f1f98', 'id': 11503}, {'uuid': '5eb46de2-9336-4b10-89e9-b31273d0d9bb', 'id': 11504}, {'uuid': '14cb9a72-9b74-4e53-abe4-c85156e78d0a', 'id': 11505}, {'uuid': 'd9bace5b-30f9-4090-a9aa-f72d7da788f2', 'id': 11506}, {'uuid': 'e40349cc-2420-4666-a1a4-f5b3f95a19e4', 'id': 11507}, {'uuid': '8b089c96-fafa-474b-95ea-22ca80d07def', 'id': 11508}, {'uuid': '27ae9be2-1e56-4431-9b36-ac23d990c3ed', 'id': 11509}, {'uuid': 'f00b00df-56a3-4bf5-854c-733f93364a56', 'id': 11510}, {'uuid': '02dd5526-e0c2-470b-9566-53482c3c4a29', 'id': 11511}, {'uuid': 'b9047ba8-e377-42b7-a0cb-05b0104563b0', 'id': 11512}, {'uuid': 'c2bd956b-f80f-4213-af5b-865518109737', 'id': 11513}, {'uuid': 'f550a619-a0b4-429b-87c8-a79d3a49099b', 'id': 11514}, {'uuid': 'c1c9ef58-2341-4ff3-9694-549e9be32ded', 'id': 11515}, {'uuid': 'b8642816-f6d1-4a4e-a8c2-cd578625f104', 'id': 11516}, {'uuid': '67a486ec-bc6a-4817-99b0-b48672a70768', 'id': 11517}, {'uuid': 'c962d0b0-5b4f-48c0-a99e-6a043ffbd7fd', 'id': 11518}, {'uuid': '906f9f0d-9b28-4cff-9f39-96aa7a8a2eba', 'id': 11519}, {'uuid': '787aed37-406d-4dfe-ab9d-ce984269095e', 'id': 11520}, {'uuid': 'bf1307b8-8d21-4a3d-b0f9-f78095745037', 'id': 11521}, {'uuid': '97cb3bca-5655-47c3-975e-4e0066285210', 'id': 11522}, {'uuid': '629195a7-9edd-4a2a-ac19-f3248a0ac7fd', 'id': 11523}, {'uuid': '16f163d3-a3f8-4c26-bcf2-debfbc89e32e', 'id': 11524}, {'uuid': '3be2bc19-e4eb-469a-9f2a-25f60b694ce8', 'id': 11525}, {'uuid': '49c77c1f-f6ed-448e-8287-f1438eb95187', 'id': 11526}, {'uuid': '28a99337-d157-4941-921a-e009db816d6a', 'id': 11527}, {'uuid': '46079823-690a-40ec-87f8-cdf7bb439e62', 'id': 11528}, {'uuid': '96a7d467-876f-4346-bf66-edfa49abb7e0', 'id': 11529}, {'uuid': '43b8a2d9-0f5f-4472-8ea6-86e593ecf893', 'id': 11530}, {'uuid': '6a1976dd-c0ff-4564-87c0-8e31d7e0145f', 'id': 11531}, {'uuid': '8914d84e-f2e2-4a6f-854a-5cbef837dea1', 'id': 11532}, {'uuid': '55da98e8-a3aa-4a3e-aba0-42855764d4d0', 'id': 11533}, {'uuid': 'a7f20397-b63a-4a6a-a139-02ead73dfa4b', 'id': 11534}, {'uuid': 'ed15b235-c22e-4974-9bd5-c825eb1d3bad', 'id': 11535}, {'uuid': 'af8be1af-3f21-495f-a9f5-a3038a0fd472', 'id': 11536}, {'uuid': '1844cb11-4bdc-48f6-aa03-d7658b47feb5', 'id': 11537}, {'uuid': '33ab888c-72b4-401a-b2d1-9794664bee4a', 'id': 11538}, {'uuid': '651cb187-af1f-4d11-9742-91f9f85a6415', 'id': 11539}, {'uuid': '1650154a-15ef-4fe3-8df2-ca6f2521f008', 'id': 11540}, {'uuid': 'c9bde34e-66a1-4a3a-a9d0-37e269f81175', 'id': 11541}, {'uuid': 'b810e809-b3ab-4733-b056-42b0deae64c7', 'id': 11542}, {'uuid': '5f2a475c-746c-4b80-9bff-2eb0dff172d0', 'id': 11543}, {'uuid': '02825286-8d31-412d-abb2-a811b1d3921e', 'id': 11544}, {'uuid': '3c777519-c39e-428c-a3da-a2f9cb0d7b07', 'id': 11545}, {'uuid': '632b7420-60b2-4115-9a4e-4088dcb5a5c7', 'id': 11546}, {'uuid': 'a91d3fe6-d462-4646-a47c-8de4605a5b68', 'id': 11547}, {'uuid': '3aed63cb-d149-436b-b016-7bf60f822049', 'id': 11548}, {'uuid': '01bfefbd-8382-4d27-92a8-ad2ba4cabc65', 'id': 11549}, {'uuid': '7fb4f1cb-5f97-47d8-8dff-9eed253bb3e8', 'id': 11550}, {'uuid': 'a2ec894d-f358-436e-9daa-8146aa03704f', 'id': 11551}, {'uuid': 'b7409a06-5137-4385-bb3c-40adb09616b3', 'id': 11552}, {'uuid': 'c0dde8ee-e0d3-41e2-9816-5260081df8e3', 'id': 11553}, {'uuid': 'dbb3cb37-9f6f-4da4-975c-2b204d041794', 'id': 11554}, {'uuid': 'c3f43a23-9257-4c07-b993-36892f3ebaba', 'id': 11555}, {'uuid': '85f6ddb6-6c53-4a8c-a678-97574bf766d1', 'id': 11556}, {'uuid': 'd5e46ae0-6718-4e9c-aa13-3a6204b702cd', 'id': 11557}, {'uuid': '2d6d9185-7d39-490f-94f2-93befde22b2d', 'id': 11558}, {'uuid': '839e7a05-0356-42af-8ff0-3bf7628e6dcd', 'id': 11559}, {'uuid': 'eabc86bc-7f8c-45d9-9676-76fee0478cad', 'id': 11560}, {'uuid': '58380383-deac-4126-8743-f3f55583a8d9', 'id': 11561}, {'uuid': '3b55f5bb-4e9e-438b-930c-9c19420d0f26', 'id': 11562}, {'uuid': '5ebd5f2f-48e6-47de-a8b1-cb865df683fa', 'id': 11563}, {'uuid': '177f523d-9398-49fd-bfa7-a0ad4eae5ba0', 'id': 11564}, {'uuid': '13525425-e40c-4c74-944f-511b7eb724d8', 'id': 11565}, {'uuid': '14787787-5704-4d2f-8b52-76a8cda0b111', 'id': 11566}, {'uuid': 'f54ba931-1598-47f7-8beb-cdad08dd3bb0', 'id': 11567}, {'uuid': '91e75098-4fb5-463c-b84a-1c11e293c133', 'id': 11568}, {'uuid': '1c23ece6-5404-4cc9-8da6-b77c4f622ebe', 'id': 11569}, {'uuid': '9925d326-5425-468e-95c8-e4c3216f0689', 'id': 11570}, {'uuid': '9214d25a-1771-4aa3-ae44-f129146c3ff2', 'id': 11571}, {'uuid': '11cdee82-edf0-4750-b6de-44d599a542dc', 'id': 11572}, {'uuid': '542796cc-7c20-41f3-a41b-14296f7b972f', 'id': 11573}, {'uuid': 'e77e1885-2ee9-4fd8-821c-dd0b0ec85337', 'id': 11574}, {'uuid': '93b3b707-402b-4382-a492-22ba63f13ec4', 'id': 11575}, {'uuid': 'b41e9657-b596-4267-90e8-2bea461bed51', 'id': 11576}, {'uuid': '17f160c5-c8e8-4f01-8451-199843b5adb1', 'id': 11577}, {'uuid': '3932b5c1-8877-4678-afa4-4f0c9e28f319', 'id': 11578}, {'uuid': 'e1eb9f83-698d-4990-baa8-49862654671c', 'id': 11579}, {'uuid': 'b8cea048-5253-4b19-b938-34ff3d3b0d52', 'id': 11580}, {'uuid': 'ec27233c-86d9-497e-9d16-a3cd3a493c6d', 'id': 11581}, {'uuid': 'acfde786-6feb-4521-964c-75272875de18', 'id': 11582}, {'uuid': '2e471b7f-f5ac-4cac-a2e5-45d01d61d2ac', 'id': 11583}, {'uuid': 'adeb9bc9-f44f-4add-b76d-ae58fdfdd0af', 'id': 11584}, {'uuid': 'e9bc17cb-71f6-4a7e-9450-c8cf9ea682a4', 'id': 11585}, {'uuid': 'e9f834d4-821f-464a-9ead-30562323b686', 'id': 11586}, {'uuid': '90ae33d5-5140-49a8-a757-3b62d69c0bfc', 'id': 11587}, {'uuid': 'd8d61a7c-05e5-4377-912e-807f3e2224b2', 'id': 11588}, {'uuid': 'e7045fdb-b539-4605-8298-0221f8810f5c', 'id': 11589}, {'uuid': '7c63c284-ac3d-4c82-b6ef-34c4941a31e2', 'id': 11590}, {'uuid': '56a45ed5-e550-48ab-b25e-57002b7e020e', 'id': 11591}, {'uuid': 'c0e1e253-9dae-4792-ac50-8377196f7b5e', 'id': 11592}, {'uuid': 'dd39125f-d48b-4534-844d-b25a020e73ae', 'id': 11593}, {'uuid': 'ab5abc4c-e199-4f0d-899b-fb43fcaac30d', 'id': 11594}, {'uuid': '7c596490-b91d-4282-94cf-c023ece94cec', 'id': 11595}, {'uuid': '74750452-fda1-42e3-b0d9-1956c54247ff', 'id': 11596}, {'uuid': '11c2260e-4814-4ed0-a40a-c5ee9c79ad20', 'id': 11597}, {'uuid': '29c478d3-9b34-4568-a087-f3f30bf7aa44', 'id': 11598}, {'uuid': '4964832d-ea89-42cf-b47b-c89b41299606', 'id': 11599}, {'uuid': '6ea49957-a40e-41fd-a15d-a1f090f05fb8', 'id': 11600}, {'uuid': 'dfbe939a-968d-444b-999c-a117c1addb25', 'id': 11601}, {'uuid': '64a44b41-5ddb-4ce5-97dc-7fba2fe3f6b6', 'id': 11602}, {'uuid': '593640b8-25e4-403c-a446-ab7bc987c049', 'id': 11603}, {'uuid': '2357a8c4-cb4a-4e94-8a8a-0cbe728224b8', 'id': 11604}, {'uuid': '25ba4f29-b940-43d4-965c-ee9a0a58bfbc', 'id': 11605}, {'uuid': '30f7492b-aed2-4d79-9803-8297ee656628', 'id': 11606}, {'uuid': '933638dc-d94b-4039-81d7-14379c5bddfe', 'id': 11607}, {'uuid': '8d043828-b8c4-4332-b4d1-46fb81b69d43', 'id': 11608}, {'uuid': '7eb4c313-8c56-42b4-a690-54312b45e7a4', 'id': 11609}, {'uuid': '5b7657d8-516f-4ed8-9615-0230d56e8961', 'id': 11610}, {'uuid': 'c3352213-7194-41c3-a323-348e97445c5e', 'id': 11611}, {'uuid': '7ca0c555-ebbb-4aa3-999d-015d69e0b150', 'id': 11612}, {'uuid': '7974c79e-6b96-4c0a-a2ac-330598b3e5e9', 'id': 11613}, {'uuid': 'd8ae65b1-7490-436e-8273-38ffc69d5306', 'id': 11614}, {'uuid': '28bc2eb6-721a-4820-bd9f-465eed3804d8', 'id': 11615}, {'uuid': '61ac93c9-f297-4323-9638-b98af295f195', 'id': 11616}, {'uuid': '542ee54f-de6a-42de-9553-392bdcdfe6cf', 'id': 11617}, {'uuid': 'a611f8b6-d2cc-402e-bbeb-8c51a85a8686', 'id': 11618}, {'uuid': '9b88ece2-2369-4abd-9dc6-17dc7a218490', 'id': 11619}, {'uuid': 'fb9b6097-59e7-48d1-9a24-97b0aa4a34cd', 'id': 11620}, {'uuid': 'a971556a-5ed3-4deb-a59d-c4e9dcefda19', 'id': 11621}, {'uuid': 'dba9e575-2484-4f9b-a851-5a2d509c9075', 'id': 11622}, {'uuid': '4024553a-75a8-44c0-bfa6-7fc543f66261', 'id': 11623}, {'uuid': 'cb0e9e95-0131-41d7-92ae-58c922e33e53', 'id': 11624}, {'uuid': 'a9bcb9cb-e929-401d-931c-2bd3f9410984', 'id': 11625}, {'uuid': '6eb039f3-2b65-4051-8885-79f715eba503', 'id': 11626}, {'uuid': 'b23378cf-75ce-4c83-92a1-624ee1ce8a9b', 'id': 11627}, {'uuid': '2397460e-f93e-4bcd-9441-4945202866de', 'id': 11628}, {'uuid': 'ec9c614c-ba73-4bb7-ab79-ee4fe27cf86f', 'id': 11629}, {'uuid': '8fe3ddbe-3ba2-492d-a6dd-e10f6b861bba', 'id': 11630}, {'uuid': 'fd3c00da-f12e-4229-9c3d-31527a726787', 'id': 11631}, {'uuid': '0b6877ef-bcae-4066-a592-eedbd4256ced', 'id': 11632}, {'uuid': '4aa7ec70-fb2a-4ae7-bbb2-c9e7a7c40cf0', 'id': 11633}, {'uuid': '040eff30-5ae8-4f1b-ac2c-2e45d9d65bd6', 'id': 11634}, {'uuid': 'b93b8299-4ad9-4e6a-b6f6-1625a5763737', 'id': 11635}, {'uuid': '14c679f4-1224-43b1-af2a-8c4351ee0182', 'id': 11636}, {'uuid': '922ccae0-cb13-4dab-a741-c19ae2950e83', 'id': 11637}, {'uuid': '68390381-837e-4a96-93d9-69d5ed6d37fc', 'id': 11638}, {'uuid': 'a79aad0b-3636-492f-b710-844aa943caea', 'id': 11639}, {'uuid': '279fde4c-6970-42b1-b321-f9c43966a012', 'id': 11640}, {'uuid': '1bf191cc-61f1-46c0-8868-2ca047126726', 'id': 11641}, {'uuid': '0b0836d2-09a1-4f44-9d4e-e31396cfd367', 'id': 11642}, {'uuid': '12599b53-714e-4695-b990-8cff57423608', 'id': 11643}, {'uuid': '5e1a7bd4-31da-42b0-abce-c1d671194b27', 'id': 11644}, {'uuid': 'd249b031-d17a-4b88-819e-9ac9a5b795ab', 'id': 11645}, {'uuid': '31d336f8-66ac-48ba-9b52-ef8f4683a5e1', 'id': 11646}, {'uuid': 'b39ee169-0e70-4346-bad3-02804b857192', 'id': 11647}, {'uuid': '98ce91dc-ed7a-4d7f-bc60-9a74ab4ee3f4', 'id': 11648}, {'uuid': '85f799ee-06fd-4ec1-9b8d-a5f13e5b736f', 'id': 11649}, {'uuid': '8710051d-635b-468c-9b4b-ecc2ac86027a', 'id': 11650}, {'uuid': '9875269d-db3b-4f8c-a4ba-a12ee5219709', 'id': 11651}, {'uuid': '602339ca-fc24-4ed7-bcdf-3ee574813fb9', 'id': 11652}, {'uuid': '5b0ada9c-6614-4664-b899-46fa2d9df81c', 'id': 11653}, {'uuid': '6c753634-98a1-4d1e-a7e2-43e21ad2bc5d', 'id': 11654}, {'uuid': 'cc853258-05b7-4e7d-96a3-46eab29b6f8e', 'id': 11655}, {'uuid': '4ac6f86d-4c4c-4d17-8430-94c31f80cc35', 'id': 11656}, {'uuid': 'fe1f215a-5a25-4de5-aa56-114cbe354480', 'id': 11657}, {'uuid': '1c731ce7-554b-4e55-b39c-8bb29b98372c', 'id': 11658}, {'uuid': '48d78e7a-49ed-4c02-8887-f039bfd50864', 'id': 11659}, {'uuid': '8ba3d713-07e1-4dd0-a731-34f3cb4d0b75', 'id': 11660}, {'uuid': '762596fb-46dc-452a-8f9c-ada83fee8e5d', 'id': 11661}, {'uuid': 'a906a365-9598-4aee-9d6b-5ca5410c2470', 'id': 11662}, {'uuid': '86f6026f-90af-4ae3-b24a-415cf85b4775', 'id': 11663}, {'uuid': '14e18985-d6fb-4b17-82f7-dcc6233f1e0c', 'id': 11664}, {'uuid': '5e69c64c-4eb9-4def-83a2-f9b067833777', 'id': 11665}, {'uuid': 'dfdf1395-3097-4290-97b2-8ab13aef440d', 'id': 11666}, {'uuid': '943b5b5c-343a-4eb7-8a20-89e489b6935d', 'id': 11667}, {'uuid': 'f2f8a899-6add-4500-9402-72d0466bbd7b', 'id': 11668}, {'uuid': 'ff075a4b-4a26-442c-bac1-88864bb239bf', 'id': 11669}, {'uuid': '2254921a-d89a-4714-a73f-846cbb24f2d5', 'id': 11670}, {'uuid': 'fac2ee83-8d83-419a-b362-8a31619e6192', 'id': 11671}, {'uuid': '8774825c-0fb2-4f6f-b256-61f764f52f97', 'id': 11672}, {'uuid': '72a4711c-00a3-45f1-b4b0-8f6d20ccb05e', 'id': 11673}, {'uuid': '4523263a-5e8f-4b92-80f7-9e4b45dbce6c', 'id': 11674}, {'uuid': '483a0246-2dbd-4d11-954c-c19efb4282f6', 'id': 11675}, {'uuid': '43471a76-f86d-4dec-b62d-e02da9266b86', 'id': 11676}, {'uuid': '03f81a45-be66-4b09-8d9d-e732aaa37121', 'id': 11677}, {'uuid': '9f5f2266-1384-4462-94a1-9aea3ee1e93e', 'id': 11678}, {'uuid': 'f604b435-5b4c-4388-b6c1-a8b70a387c6d', 'id': 11679}, {'uuid': '719167ab-1173-4e85-b907-a3820a606701', 'id': 11680}, {'uuid': '77a4e880-d764-48cb-9be2-0adf162786f3', 'id': 11681}, {'uuid': '49b53ab2-00f4-4b40-95ac-4fcb6740c52c', 'id': 11682}, {'uuid': '8d037ab8-af88-48cb-a687-6d92961aafbd', 'id': 11683}, {'uuid': '2fd01406-0745-4f28-9ccc-a9c422c9861a', 'id': 11684}, {'uuid': '2ab95a34-332d-485a-8141-f9b46671634d', 'id': 11685}, {'uuid': 'e3f1933a-d90c-43e8-983f-9877006395e1', 'id': 11686}, {'uuid': '87b039b7-dd83-4f60-b115-6a8365705594', 'id': 11687}, {'uuid': '584a7daf-1db5-4de8-b5c3-26fb83e30bae', 'id': 11688}, {'uuid': '6d257a99-5671-4cf5-b2c8-7b1b654e7cb5', 'id': 11689}, {'uuid': 'c4a645df-4871-4c37-a1ab-82b30ce0f02e', 'id': 11690}, {'uuid': 'c0785723-6a8d-4949-abc0-d8772fac3970', 'id': 11691}, {'uuid': 'fd1bb0cb-c16c-451a-8f5b-e083789a8cf1', 'id': 11692}, {'uuid': 'e7cc49ae-4839-47da-a3fd-b0a7e6b01ae0', 'id': 11693}, {'uuid': '9bce28dd-0550-4067-8ca2-a1d50f13f9cb', 'id': 11694}, {'uuid': 'fa3d1bd8-7206-4ce2-a6a9-1f0fa1785cf9', 'id': 11695}, {'uuid': 'af638a75-b791-4c4e-a572-4fcc1d16dfdb', 'id': 11696}, {'uuid': 'a6afb02a-9256-497b-b574-ebc0b2cad525', 'id': 11697}, {'uuid': '7b8249de-c15c-492b-984d-f234b0ad9aa2', 'id': 11698}, {'uuid': '1b51c273-55c0-462c-a779-c8c84d196dc4', 'id': 11699}]

In [4]:

import os
import django
from langfuse.decorators import observe, langfuse_context

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'xiaoxi.settings_prod')  # 替换为你的settings文件路径
django.setup()

from xiaoxi_ai.database.mysql import ai_knowledge_mapper
ai_knowledge_mapper.update_ai_knowledge_uuid(le)